In [ ]:
!pip install tensorflow

In [310]:
import cv2
import os
import random
import tensorflow as tf
from shutil import copyfile
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img

In [279]:
data_dirs = ['Pepper__bell___Bacterial_spot', 'Potato___healthy', 'Pepper__bell___healthy']

In [280]:
source_path = 'sources'

DATA_CLASSES = ['Bell Pepper Bacterial Spot', 'Potato Healthy', 'Bell Pepper Healthy']

for i, dir in enumerate(data_dirs):
    class_dir = os.path.join(source_path, dir)
    print(f'There are {len(os.listdir(class_dir))} images of {DATA_CLASSES[i]}')

There are 997 images of Bell Pepper Bacterial Spot
There are 152 images of Potato Healthy
There are 1478 images of Bell Pepper Healthy


In [281]:
root_binary = 'binary_data'

os.makedirs(os.path.join(root_binary, 'train'))
# os.makedirs(os.path.join(root_binary, 'validation'))
os.makedirs(os.path.join(root_binary, 'testing'))

os.makedirs(os.path.join(root_binary, 'train/bell_pepper_bac'))
os.makedirs(os.path.join(root_binary, 'train/potato_healthy'))

# os.makedirs(os.path.join(root_binary, 'validation/bell_pepper_bac'))
# os.makedirs(os.path.join(root_binary, 'validation/potato_healthy'))

os.makedirs(os.path.join(root_binary, 'testing/bell_pepper_bac'))
os.makedirs(os.path.join(root_binary, 'testing/potato_healthy'))

In [296]:
def split_data(SOURCE, TRAINING, TESTING, TRAINING_SPLIT, TESTING_SPLIT):
    filtered_files = []

    for file_name in os.listdir(SOURCE):
        file_path = os.path.join(SOURCE, file_name)
        
        if os.path.getsize(file_path):
            filtered_files.append(file_name)
        else:
            print('{} is zero length, so ignoring.'.format(file_name))
  
    # Split the files
    random_files = random.sample(filtered_files, len(filtered_files))
    split_train = int(TRAINING_SPLIT * len(filtered_files))
    # split_val = int(VALIDATION_SPLIT * len(filtered_files))
    # split_test = int(TESTING_SPLIT * len(filtered_files))
    
    # print(split_train, split_val, split_test)

    training_files = random_files[:split_train]
    # val_files = random_files[split_val:split_test-1]
    test_files = random_files[split_train:]

    # Copy the files
    for file_name in training_files:
        copyfile(os.path.join(SOURCE, file_name), os.path.join(TRAINING, file_name))

    # for file_name in val_files:
    #     copyfile(os.path.join(SOURCE, file_name), os.path.join(VALIDATION, file_name))

    for file_name in test_files:
        copyfile(os.path.join(SOURCE, file_name), os.path.join(TESTING, file_name))

In [297]:
BELL_PEPPER_BAC_DIR = "sources/" + data_dirs[0]
POTATO_HEALTHY_DIR = "sources/" + data_dirs[1]

TRAINING_DIR = "binary_data/train/"
# VAL_DIR = "binary_data/validation/"
TEST_DIR = "binary_data/testing/"

TRAINING_BELL_PEPPER_BAC_DIR = os.path.join(TRAINING_DIR, "bell_pepper_bac/")
# VAL_BELL_PEPPER_BAC_DIR = os.path.join(VAL_DIR, "bell_pepper_bac/")
TEST_BELL_PEPPER_BAC_DIR = os.path.join(TEST_DIR, "bell_pepper_bac/")

TRAINING_POTATO_HEALTHY_DIR = os.path.join(TRAINING_DIR, "potato_healthy/")
# VAL_POTATO_HEALTHY_DIR = os.path.join(VAL_DIR, "potato_healthy/")
TEST_POTATO_HEALTHY_DIR = os.path.join(TEST_DIR, "potato_healthy/")

# Empty directories in case you run this cell multiple times
if len(os.listdir(TRAINING_BELL_PEPPER_BAC_DIR)) > 0:
    for file in os.scandir(TRAINING_BELL_PEPPER_BAC_DIR):
        os.remove(file.path)
if len(os.listdir(TRAINING_POTATO_HEALTHY_DIR)) > 0:
    for file in os.scandir(TRAINING_POTATO_HEALTHY_DIR):
        os.remove(file.path)

# if len(os.listdir(VAL_BELL_PEPPER_BAC_DIR)) > 0:
#     for file in os.scandir(VAL_BELL_PEPPER_BAC_DIR):
#         os.remove(file.path)
# if len(os.listdir(VAL_POTATO_HEALTHY_DIR)) > 0:
#     for file in os.scandir(TRAINING_POTATO_HEALTHY_DIR):
#         os.remove(file.path)        
  
if len(os.listdir(TEST_BELL_PEPPER_BAC_DIR)) > 0:
    for file in os.scandir(TEST_BELL_PEPPER_BAC_DIR):
        os.remove(file.path)
if len(os.listdir(TEST_POTATO_HEALTHY_DIR)) > 0:
    for file in os.scandir(TEST_POTATO_HEALTHY_DIR):
        os.remove(file.path)

        
# Define proportion of images used for training
split_train_size = .9

# Run the function
# NOTE: Messages about zero length images should be printed out
split_data(BELL_PEPPER_BAC_DIR, TRAINING_BELL_PEPPER_BAC_DIR, TEST_BELL_PEPPER_BAC_DIR, split_train_size, split_test_size)
split_data(POTATO_HEALTHY_DIR, TRAINING_POTATO_HEALTHY_DIR, TEST_POTATO_HEALTHY_DIR, split_train_size, split_test_size)


# Check that the number of images matches the expected output
print(f"\n\nThere are {len(os.listdir(TRAINING_BELL_PEPPER_BAC_DIR))} images of {DATA_CLASSES[0]} for training")
print(f"There are {len(os.listdir(TRAINING_POTATO_HEALTHY_DIR))} images of {DATA_CLASSES[1]} for training")

# print(f"There are {len(os.listdir(VAL_BELL_PEPPER_BAC_DIR))} images of {DATA_CLASSES[0]} for validation")
# print(f"There are {len(os.listdir(VAL_POTATO_HEALTHY_DIR))} images of {DATA_CLASSES[1]} for validation")

print(f"There are {len(os.listdir(TEST_BELL_PEPPER_BAC_DIR))} images of {DATA_CLASSES[0]} for testing")
print(f"There are {len(os.listdir(TEST_POTATO_HEALTHY_DIR))} images of {DATA_CLASSES[1]} for testing")



There are 897 images of Bell Pepper Bacterial Spot for training
There are 136 images of Potato Healthy for training
There are 100 images of Bell Pepper Bacterial Spot for testing
There are 16 images of Potato Healthy for testing


In [298]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
    # Instantiate the ImageDataGenerator class 
    train_datagen = ImageDataGenerator(rescale = 1.0 / 255.,
                                     rotation_range = 40,
                                     width_shift_range = .2,
                                     height_shift_range = .2,
                                     shear_range = .2,
                                     zoom_range = .2,
                                     horizontal_flip = True,
                                     fill_mode = 'nearest')

    # Pass in the appropriate arguments to the flow_from_directory method
    train_generator = train_datagen.flow_from_directory(directory = TRAINING_DIR,
                                                      batch_size = 64,
                                                      class_mode = 'categorical',
                                                      target_size = (150, 150))

    # Instantiate the ImageDataGenerator class 
    validation_datagen = ImageDataGenerator(rescale = 1.0 / 255.,
                                          rotation_range = 40,
                                          width_shift_range = .2,
                                          height_shift_range = .2,
                                          shear_range = .2,
                                          zoom_range = .2,
                                          horizontal_flip = True,
                                          fill_mode = 'nearest')

    # Pass in the appropriate arguments to the flow_from_directory method
    validation_generator = validation_datagen.flow_from_directory(directory = VALIDATION_DIR,
                                                                batch_size = 64,
                                                                class_mode = 'categorical',
                                                                target_size = (150, 150))

    return train_generator, validation_generator

In [299]:
# Test your generators
train_generator, validation_generator = train_val_generators(TRAINING_DIR, TEST_DIR)

Found 1033 images belonging to 2 classes.
Found 116 images belonging to 2 classes.


In [339]:
# model = Sequential()

In [351]:
model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=(256,256, 3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation='relu', input_shape=(256,256, 3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3), activation='relu', input_shape=(256,256, 3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(128, (3,3), activation='relu', input_shape=(256,256, 3)))
model.add(MaxPooling2D(2,2))

model.add(Conv2D(64, (3,3), activation='relu', input_shape=(256,256, 3)))
model.add(MaxPooling2D(2,2))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))


In [352]:
model.summary()

Model: "sequential_27"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_109 (Conv2D)         (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_96 (MaxPooli  (None, 127, 127, 32)      0         
 ng2D)                                                           
                                                                 
 conv2d_110 (Conv2D)         (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_97 (MaxPooli  (None, 62, 62, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_111 (Conv2D)         (None, 60, 60, 128)       73856     
                                                                 
 max_pooling2d_98 (MaxPooli  (None, 30, 30, 128)     

In [353]:
# Compile the model
model.compile(optimizer=Adam(), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

In [354]:
# Train the model
history = model.fit(
    train_generator,
    epochs=5,  
    validation_data=validation_generator,
    verbose=1
)

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node sequential_27/dense_48/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/steven/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/Users/steven/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/Users/steven/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start

  File "/Users/steven/anaconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/Users/steven/anaconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/Users/steven/anaconda3/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/Users/steven/anaconda3/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/Users/steven/anaconda3/lib/p